In [71]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

In [61]:
df=pd.read_csv(r"Final_dataset.csv")
df.head()

,Index,Review,Cleaned_Review,Cleaned_Name,Severity,Urgency,Customer_Satisfaction,Review_Days,Response_Days,Predicted_Category,Service Issue,Food Quality,Atmosphere,Value for Money,Hygiene,Food Options
0,0,"Stop eating at this place, I have visited bang...",stop eating place visited bangalores nd punes ...,pramod kumar,High,Urgent,No Response,180,-1,['Food Quality' 'Atmosphere' 'Hygiene'],0,1,1,0,1,0
1,1,Food 2/5\nService 2/5\nAmbience 2/5 …,food service ambience,abhinav deep,Medium,Non-Urgent,High Satisfaction,365,365,['Atmosphere' 'Service Issue' 'Food Quality' '...,1,1,1,0,0,1
2,2,Idiotic varieties for the price they have char...,idiotic varieties price charged varieties boil...,vijay nammi,High,Urgent,No Response,30,-1,['Food Options'],0,0,0,0,0,1
3,3,"I am posting this live now, this is one of the...",posting live one worst places dont visit pathe...,surya ajay,High,Urgent,High Satisfaction,365,365,['Service Issue' 'Food Quality' 'Atmosphere'],1,1,1,0,0,0
4,4,"We are pure vegetarians, I ordered veg biryani...",pure vegetarians ordered veg biryani swiggy go...,sai hithesh,Low,Non-Urgent,No Response,180,-1,['Food Quality'],0,1,0,0,0,0


In [62]:
categories_to_add = ['Service Issue', "Food Quality", "Atmosphere", "Value for Money", "Hygiene","Food Options"]

In [63]:
# Device configuration (use CUDA if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [64]:
# Project path for saving and loading the model
project_path = "E:/Gradious_Final_Project"

In [65]:
# Define the categories for multi-label classification
categories_to_add = ['Service Issue', 'Food Quality', 'Food Options', 'Atmosphere', 'Value for Money', 'Hygiene']

# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Cleaned_Review'], df[categories_to_add], test_size=0.2, random_state=42
)

In [66]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [67]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

In [68]:

# Convert to tensors
train_labels = torch.tensor(train_labels.values).float()
test_labels = torch.tensor(test_labels.values).float()

In [69]:
# 2. Dataset and DataLoader with Batch Size 2
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)


In [70]:
train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)

In [73]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [74]:
# 3. Load Model and Optimizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(categories_to_add)).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
